In [21]:
import matplotlib.pyplot as plt
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom, propose_random_flip
from gerrychain.tree import recursive_tree_part
from gerrychain.metrics import mean_median, efficiency_gap, polsby_popper, partisan_gini
from functools import (partial, reduce)
import pandas
import geopandas as gp
import numpy as np
import networkx as nx
import pickle
import seaborn as sns
import pprint
import operator
import scipy

from gerrychain.accept import always_accept
from gerrychain.constraints import (Validator, single_flip_contiguous,
within_percent_of_ideal_population, UpperBound)

import json

import random

In [2]:
graph = Graph.from_json("BG22.json")

In [4]:
tot_pop = 0

for i in range(len(graph.nodes)):
    tot_pop += graph.nodes[i]['TOTPOP']
print(tot_pop)
ideal_population_LO = int(tot_pop/39)


4533372


In [8]:
graph.nodes[0]

{'boundary_node': False,
 'area': 383781.2515734935,
 'STATEFP10': '22',
 'COUNTYFP10': '071',
 'TRACTCE10': '011700',
 'BLKGRPCE10': '4',
 'GEOID10': '220710117004',
 'NAMELSAD10': 'Block Group 4',
 'MTFCC10': 'G5030',
 'FUNCSTAT10': 'S',
 'ALAND10': 383782,
 'AWATER10': 0,
 'INTPTLAT10': '+29.9337991',
 'INTPTLON10': '-090.1179266',
 'GISJOIN': 'G22007100117004',
 'Shape_area': 383781.251573,
 'Shape_len': 2798.1782813,
 'YEAR': '2010',
 'REGIONA': '3',
 'DIVISIONA': '7',
 'STATE': 'Louisiana',
 'STATEA': '22',
 'COUNTY': 'Orleans Parish',
 'COUNTYA': '71',
 'TRACTA': '11700',
 'BLKGRPA': '4',
 'NAME': 'Block Group 4',
 'URBPOP': '967',
 'RURALPOP': '0',
 'TOTPOP': 967,
 'NH_WHITE': 884,
 'NH_BLACK': 19,
 'NH_AMIN': 0,
 'NH_ASIAN': 20,
 'NH_NHPI': 0,
 'NH_OTHER': 0,
 'NH_2MORE': 9,
 'HISP': 35,
 'H_WHITE': 28,
 'H_BLACK': 0,
 'H_AMIN': 0,
 'H_ASIAN': 0,
 'H_NHPI': 0,
 'H_OTHER': 6,
 'H_2MORE': 1,
 'VAP': 769,
 'HVAP': 27,
 'WVAP': 706,
 'BVAP': 13,
 'AMINVAP': 0,
 'ASIANVAP': 19,
 'N

In [5]:
LO_updaters = {"population": updaters.Tally("TOTPOP", alias="population"),
           "bvap":updaters.Tally("BVAP",alias='bvap'),
            'vap':updaters.Tally("VAP",alias='vap'),
           }


In [ ]:
def increase_opportunity_districts (proposal_part, prev_opp_dist):
    for district in proposal_part

In [14]:
seed_part = recursive_tree_part(graph, range(39), pop_col="TOTPOP",
                               pop_target=ideal_population_LO,
                               epsilon=0.01, node_repeats=1)
LO_seed = GeographicPartition(graph,assignment=seed_part,updaters=LO_updaters)


#enacted_part = GeographicPartition(graph, assignment="_16", updaters=updaters)

In [16]:
popbound=within_percent_of_ideal_population(LO_seed,.1)

In [25]:
recom_chain = MarkovChain(seed_part, Validator([popbound]),accept=always_accept,
                          initial_state=LO_seed,total_steps=10)

In [29]:
for part in recom_chain:
    print(part.parts)

{0: frozenset({1280, 1589, 1591, 569, 1850, 571, 572, 1341, 1342, 1343, 1344, 577, 578, 579, 1345, 581, 582, 583, 1348, 1349, 1351, 3154, 2386, 85, 86, 87, 88, 89, 90, 3162, 91, 3166, 1909, 933, 934, 935, 936, 950, 951, 952, 953, 954, 713, 714, 716, 1999, 721, 722, 723, 2004, 3033, 3034, 3035, 988, 3037, 990, 1759, 3036, 1761, 1762, 1763, 1764, 1765, 1766, 1767, 3040, 3041, 487, 235, 488, 489, 491, 492, 493, 494, 495, 1522, 1526, 247, 1016, 1017, 251}), 1: frozenset({644, 2826, 145, 146, 151, 792, 153, 152, 155, 1820, 1821, 1949, 1053, 156, 3102, 1954, 1955, 3107, 1958, 1064, 1961, 2734, 688, 1074, 1077, 1726, 1613, 598, 599, 1368, 995, 740, 997, 744, 1384, 1386, 1385, 1129, 746, 1132, 879, 745, 3057, 1137, 749, 1134, 109, 110, 111, 1651, 881, 1653, 882, 2815}), 2: frozenset({3073, 1026, 1027, 3076, 1025, 1541, 1028, 1284, 1033, 1286, 1527, 3087, 3090, 2842, 1056, 1057, 1824, 1825, 1828, 1829, 1835, 3117, 1840, 1842, 1846, 1847, 3064, 2381, 2384, 2385, 1618, 1619, 1620, 3156, 1623, 162

TypeError: 'dict' object is not callable